In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import re
import gc
import nltk
from nltk.tokenize import RegexpTokenizer
import psutil
import numpy as np
import pandas as pd
from numpy import zeros
from numpy import array
from numpy.random import choice

#nltk.download('punkt')

def merge_entity_metadata_by_id():
  entity_ids_df = pd.read_csv(os.path.join(DATA_DIR, "entity_ids.csv"), 
                              usecols=['entity', 'id'],
                              index_col='id',
                             )
  #print(entity_ids_df)
  
  entity_metadata_df = pd.read_csv(os.path.join(DATA_DIR, "entity_metadata.csv"),
                                  )
  entity_metadata_df = entity_metadata_df.rename(columns={'entity':'id', 'humanreadablererpresentation': 'metadata'})
  #print(entity_metadata_df)
  
  # merge data 
  id_entity_metadata_df = pd.merge(entity_ids_df,entity_metadata_df, on=['id'], how= 'left')
#   print(id_entity_metadata_df.head(10))
  id_entity_metadata_df = id_entity_metadata_df.set_index('entity').filter(regex="[<][a-zA-Z]+[>]", axis=0)
#   id_entity_metadata_df = id_entity_metadata_df.set_index('entity').filter(regex="[<][a-z]+[>]", axis=0)
  return id_entity_metadata_df
  
def merge_entity_metatype_by_id():
  entity_types_df = pd.read_csv(os.path.join(DATA_DIR, "entity_types.csv"),
                               )
  entity_types_df = entity_types_df.rename(columns={'entity':'id'})
  #print(entity_types_df)
  
  type_ids_df = pd.read_csv(os.path.join(DATA_DIR, "type_ids.csv"),
                            usecols=['id', 'metatype'],
                           )
  type_ids_df = type_ids_df.rename(columns={'id':'type'})
  #print(type_ids_df)
  
  id_type_metatype_df = pd.merge(entity_types_df,type_ids_df, on=['type'], how= 'left')
  return id_type_metatype_df

def merge_news_header_fulltext():
  # get_text_id_header_df
  id_header_file = os.path.join(DATA_DIR, "full-news-10000.txt")
  f = open(id_header_file)
  lines = f.readlines()
  f.close
  id_header_dict = {}
  for line in lines:
    text_id = line.split(",")[0]
    header = line.split(',,"')[-1]
    if text_id not in id_header_dict:
      header = header.rstrip('"\n')
      id_header_dict[text_id] = header
#   print(len(id_header_dict))
  id_header_df = pd.DataFrame(list(id_header_dict.items()), columns=['id', 'header'])
#   print(id_header_df.head(10))
  
  # get_id_fulltext_df
  id_fulltext_file = os.path.join(DATA_DIR, "text.txt")
  f = open(id_fulltext_file)
  lines = f.readlines()
  f.close()
  id_fulltext_dict = {}
  for index, line in enumerate(lines):
    if index % 2 == 0:
      text_id = line.rstrip("\n")
    elif text_id not in id_fulltext_dict:
      id_fulltext_dict[text_id] = line
#   print(len(id_fulltext_dict))
  id_fulltext_df = pd.DataFrame(list(id_fulltext_dict.items()), columns=['id', 'fulltext'])
#   print(id_fulltext_df.head(10))
  
  #merge_header_fulltext_by_id
  id_header_fulltext_df = pd.merge(id_fulltext_df,id_header_df, on=['id'], how= 'left')
  return id_header_fulltext_df
  
# text to list of tokenized sents
def text_to_sents(text_list):
  sents = []
  for text in text_list:
    # sentence split
    sent_text = nltk.sent_tokenize(text)
    for sentence in sent_text:
      # word tokenization
      tokenizer = RegexpTokenizer(r'[a-zA-Z]\w+\'?\w*')
      tokenized_sent = tokenizer.tokenize(sentence)
      tokenized_sent = [word.lower() for word in tokenized_sent]
      sents.append(tokenized_sent)
  return sents

# word frequency 
def sents_processing(sents):
  # build new dict to count words in poems
  allWords = {} 
  for sent in sents:
    for word in sent:
      if word not in allWords:
        #print(word)
        allWords[word] = 1
      else:
        allWords[word] = allWords[word]+ 1
  # delete words which less than 2 times used in poems
  erase = []
  for key in allWords:
    if allWords[key] < 5:
        erase.append(key)
  #print(erase)
  allWords['<oov/>'] = 0
  for key in erase:
    allWords['<oov/>'] = allWords['<oov/>']+ allWords[key]
    del allWords[key]
  # sort allWords by counting times: key:word, value:count
  wordPairs = sorted(allWords.items(), key = lambda x: -x[1])
  print(wordPairs)
  words, a= zip(*wordPairs)
  # word to ID, most frequently used word is assigned with a short id
  word2int = dict(zip(words, range(len(words)))) 
#   print(word2int)
  # ID to word
  int2word = dict(zip(word2int.values(), word2int.keys()))
  #print(reverse_dictionary)
  return allWords, word2int, int2word

id_entity_metadata_df = merge_entity_metadata_by_id()
# print(id_entity_metadata_df)
entity_list = id_entity_metadata_df['metadata'].tolist()
# print(len(entity_list))
# print(entity_list[:10])
chars_list = word_to_chars_list(entity_list)
print(len(chars_list)) # 358947
print(chars_list[:10])

chars_dict, char2int, int2char = chars_list_processing(chars_list)
print(char2int)
print(len(char2int))

# print(chr(96))

# id_type_metatype_df = merge_entity_metatype_by_id()
# print(id_type_metatype_df.head(5))

# id_entity_metadata_metatype_df = pd.merge(id_type_metatype_df, id_entity_metadata_df, on=['id'], how='left')
# print(id_entity_metadata_metatype_df.head(5))
# id_header_fulltext_df = merge_news_header_fulltext()
# print(id_header_fulltext_df.head(10))

# header_list = np.array(id_header_fulltext_df['header']).tolist()
# fulltext_list = np.array(id_header_fulltext_df['fulltext']).tolist()
# print(len(header_list))
# print(header_list[:10])

# all_text_list = header_list + fulltext_list
# sents = text_to_sents(all_text_list)

# print(len(sents))
# print(sents[:10])

# allWords, word2int, int2word = sents_processing(sents)
# print(len(word2int))
# print(word2int)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


358947
[['A', 'a', 'a', 'a', 'b', 'a', '</END>'], ['A', 'a', 'a', 'g', 'e', 's', '</END>'], ['A', 'a', 'a', 'j', 'i', 'a', 'o', '</END>'], ['A', 'a', 'a', 'r', 't', 'a', 'l', 'i', '</END>'], ['A', 'a', 'b', 'a', '</END>'], ['A', 'a', 'b', 'a', 'v', 'a', 'n', 'a', 'n', '</END>'], ['A', 'A', 'B', 'B', '</END>'], ['A', 'a', 'b', 'e', 'n', 'r', 'a', 'a', '</END>'], ['A', 'a', 'b', 'e', 'y', '</END>'], ['A', 'a', 'b', 'h', 'a', 'r', 'a', 'n', 'a', 'c', 'h', 'a', 'r', 't', 'h', 'u', '</END>']]
[('a', 361648), ('</END>', 358947), ('e', 245987), ('i', 219692), ('o', 198300), ('r', 187215), ('n', 185985), ('l', 142470), ('t', 129158), ('s', 126723), ('u', 113323), ('h', 99443), ('d', 77411), ('m', 72188), ('c', 71620), ('g', 57729), ('p', 51213), ('k', 51152), ('y', 49626), ('b', 41598), ('S', 37637), ('A', 31119), ('M', 30354), ('C', 29051), ('K', 28309), ('B', 28138), ('P', 27683), ('v', 27039), ('T', 24469), ('z', 22914), ('w', 21071), ('D', 18782), ('G', 17097), ('L', 17093), ('f', 16990), 

In [0]:
import keras
import keras.backend as K
from keras.utils import to_categorical
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Embedding, Activation
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
########################################## Params ##########################################

MAX_LEN = 3
VOCAB_SIZE = 53
# totol news count 10000
START_INDEX = 0 # start index
END_INDEX = 10 # trainig number

# model building
INIT_LR = 0.001 # init learning rate
N_STEP = 10 
BATCH_SIZE = 16
LAYERS_DIM = [700, 700]
EPOCHS = 15 #training_epochs number

########################################## Embed Paths ##########################################

EMBEDDING_DIM = 100
EMBED_DIR = "drive/My Drive/DATA/" # embedding file
glove_embedding = "glove.6B.{:d}d.txt".format(EMBED_DIM)

########################################## Data Prepare ##########################################

def char_encoding(chars_list):
  chars_integer = []
  for chars in chars_list:
    char_integer = []
    for char in chars:
      char_integer.append(char2int[char])
    chars_integer.append(char_integer)
  return chars_integer

def N_gram_processing(chars, N):
  N_gram_data=[]
  N_gram_label=[]
  for i in range(len(chars)-N):
    N_gram = chars[i:i+N+1]
    N_gram_data.append(tuple(N_gram[:-1]))
    N_gram_label.append(N_gram[-1])
  return N_gram_data, N_gram_label

def get_train_data(chars_list, MAX_LEN):
  print(chars_list[:1])
  chars_list = char_encoding(chars_list)
  print(chars_list[:1])
  datas = []
  labels = []
  for chars in chars_list:
    data, label=N_gram_processing(chars, MAX_LEN)
    datas = datas+list(data)
    labels = labels+label
  X_data = np.empty((len(labels), MAX_LEN), dtype = int)
  y_label = np.empty((len(labels),VOCAB_SIZE), dtype = int)
  for index, _ in enumerate(labels):
    X_data[index, : ] = [int(x) for x in list(datas[index])]
    y_label[index] = to_categorical(labels[index], VOCAB_SIZE)
  #del datas, labels
  return X_data, y_label

def get_embedding_matrix(dictionary):
  embedding_matrix = zeros((VOCAB_SIZE, EMBEDDING_DIM), dtype=float)
  for i, key in enumerate(dictionary.keys()):
    embedding_vector = [float(w) for w in np.random.rand(1, EMBEDDING_DIM)[0]]
    if embedding_vector is not None: 
      embedding_matrix[i] = embedding_vector
  return embedding_matrix


def lr_scheduler(epoch):
  if epoch == 0:
    K.set_value(model.optimizer.lr,INIT_LR)
    print("start lr set to {}".format(K.get_value(model.optimizer.lr)))
  # every 10 step reduce lr to 0.1*lr
  elif epoch % N_STEP == 0 and epoch != 0:
    lr = K.get_value(model.optimizer.lr)
    K.set_value(model.optimizer.lr, lr * 0.1)
    print("lr changed to {}".format(K.get_value(model.optimizer.lr)))
  return K.get_value(model.optimizer.lr)

train_data, train_label = get_train_data(chars_list[:END_INDEX], MAX_LEN)
print(train_data[:10])
print(train_label[:10])

embedding_matrix = get_embedding_matrix(char2int)
print(embedding_matrix)

print("finished data preparation !!! ")

model = Sequential()
# input Layer
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_LEN, trainable=True))
# Hidden Layer
# model.add(LSTM(Layer1_dim, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(LAYERS_DIM[0], return_sequences = True, dropout = 0.2, recurrent_dropout=0.2))
model.add(Activation('relu')) 
# model.add(LSTM(LAYERS_DIM[0], return_sequences = True, dropout = 0.3, recurrent_dropout=0.3))
# model.add(Activation('relu'))
model.add(LSTM(LAYERS_DIM[1], dropout= 0.2, activation='relu'))
# Output Layer
model.add(Dense(VOCAB_SIZE, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

# fit the model
lr_reduce = keras.callbacks.LearningRateScheduler(lr_scheduler)
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
# mc = ModelCheckpoint('hex06_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.fit(train_data, train_label, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[lr_reduce], 
#           validation_data=(valid_data, valid_label)
         )

# loss, accuracy = model.evaluate(valid_data, valid_label, batch_size=valid_num)
# print('Accuracy of Valid_data: %f' % (accuracy*100))

# serialize model to JSON
model_json = model.to_json()
with open(model2json_path, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(weights_path)
print("Saved model to disk")

[['A', 'a', 'a', 'a', 'b', 'a', '</END>']]
[[21, 0, 0, 0, 19, 0, 1]]
[[21  0  0]
 [ 0  0  0]
 [ 0  0 19]
 [ 0 19  0]
 [21  0  0]
 [ 0  0 15]
 [ 0 15  2]
 [15  2  9]
 [21  0  0]
 [ 0  0 40]]
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0

NameError: ignored